In [4]:
pip install python-docx tk

   ---------------------------------------- 0.0/244.3 kB ? eta -:--:--
   - -------------------------------------- 10.2/244.3 kB ? eta -:--:--
   ------ -------------------------------- 41.0/244.3 kB 653.6 kB/s eta 0:00:01
   ---------------------------- ----------- 174.1/244.3 kB 1.7 MB/s eta 0:00:01
   -------------------------------------- - 235.5/244.3 kB 2.4 MB/s eta 0:00:01
   ---------------------------------------- 244.3/244.3 kB 1.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import json
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
from docx import Document
import re
from datetime import datetime

class DocumentFillerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Preenchimento Automático de Documentos")
        self.root.geometry("900x700")
        
        self.documents_info = {}
        self.selected_docs = []
        self.all_fields = {}
        self.field_values = {}
        
        self.setup_ui()
        self.load_json_models()

    def setup_ui(self):
        # Frame principal com notebooks para diferentes etapas
        self.main_notebook = ttk.Notebook(self.root)
        self.main_notebook.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Página 1: Seleção de documentos
        self.docs_frame = ttk.Frame(self.main_notebook)
        self.main_notebook.add(self.docs_frame, text="1. Seleção de Documentos")
        
        # Página 2: Preenchimento de dados
        self.data_frame = ttk.Frame(self.main_notebook)
        self.main_notebook.add(self.data_frame, text="2. Preenchimento de Dados")
        
        # Página 3: Geração de documentos
        self.gen_frame = ttk.Frame(self.main_notebook)
        self.main_notebook.add(self.gen_frame, text="3. Geração de Documentos")
        
        # Configurar a página de seleção de documentos
        self.setup_docs_page()
        
    def setup_docs_page(self):
        # Label de instrução
        ttk.Label(self.docs_frame, text="Selecione os documentos a serem preenchidos:", 
                 font=("Arial", 12)).pack(anchor=tk.W, padx=20, pady=(20, 10))
        
        # Frame para checkboxes de documentos
        self.docs_checkbox_frame = ttk.Frame(self.docs_frame)
        self.docs_checkbox_frame.pack(fill=tk.BOTH, expand=True, padx=20, pady=10)
        
        # Lista de modelos de documentos disponíveis
        self.doc_vars = {}
        self.doc_checkboxes = []
        
        # Botão para avançar
        ttk.Button(self.docs_frame, text="Avançar para Preenchimento", 
                  command=self.proceed_to_data).pack(pady=20)
        
        # Botão para selecionar diretório de modelos .docx
        ttk.Button(self.docs_frame, text="Selecionar Diretório de Modelos", 
                  command=self.select_docx_dir).pack(pady=10)
                  
    def select_docx_dir(self):
        dir_path = filedialog.askdirectory(title="Selecione o diretório contendo os modelos .docx")
        if dir_path:
            # Limpar os checkboxes existentes
            for cb in self.doc_checkboxes:
                cb.destroy()
            self.doc_checkboxes = []
            self.doc_vars = {}
            
            # Procurar por arquivos .docx no diretório
            docx_files = [f for f in os.listdir(dir_path) if f.endswith('.docx')]
            
            if not docx_files:
                messagebox.showinfo("Informação", "Nenhum arquivo .docx encontrado no diretório selecionado.")
                return
                
            # Criar checkboxes para cada arquivo .docx
            for idx, docx_file in enumerate(docx_files):
                var = tk.BooleanVar()
                cb = ttk.Checkbutton(self.docs_checkbox_frame, text=docx_file, variable=var)
                cb.grid(row=idx // 2, column=idx % 2, sticky=tk.W, padx=10, pady=5)
                self.doc_vars[docx_file] = var
                self.doc_checkboxes.append(cb)
                
            # Armazenar o caminho dos documentos
            self.docx_dir = dir_path
    
    def load_json_models(self):
        # Carregar informações de campos de todos os modelos JSON
        for json_file in os.listdir():
            if json_file.endswith('_modelo.json'):
                try:
                    with open(json_file, 'r', encoding='utf-8') as f:
                        json_data = json.load(f)
                        if 'campos' in json_data:
                            self.documents_info[json_file] = json_data
                except Exception as e:
                    print(f"Erro ao carregar {json_file}: {e}")
    
    def proceed_to_data(self):
        # Verificar quais documentos foram selecionados
        self.selected_docs = [doc for doc, var in self.doc_vars.items() if var.get()]
        
        if not self.selected_docs:
            messagebox.showwarning("Aviso", "Selecione pelo menos um documento para continuar.")
            return
        
        # Identificar todos os campos necessários dos documentos selecionados
        self.identify_required_fields()
        
        # Configurar a página de preenchimento de dados
        self.setup_data_page()
        
        # Mudar para a próxima aba
        self.main_notebook.select(1)
    
    def identify_required_fields(self):
        # Limpar campos anteriores
        self.all_fields = {}
        
        # Encontrar o modelo JSON correspondente para cada documento selecionado
        for selected_doc in self.selected_docs:
            base_name = selected_doc.split('.')[0]
            json_file = f"{base_name}_modelo.json"
            
            if json_file in self.documents_info:
                fields = self.documents_info[json_file]['campos']
                for field_key, field_info in fields.items():
                    if field_key not in self.all_fields:
                        self.all_fields[field_key] = field_info
            else:
                # Se não encontrar JSON específico, tentar extrair campos do documento diretamente
                try:
                    doc_path = os.path.join(self.docx_dir, selected_doc)
                    doc = Document(doc_path)
                    text = '\n'.join([p.text for p in doc.paragraphs])
                    
                    # Extrair todos os campos entre colchetes
                    fields = re.findall(r'\[(.*?)\]', text)
                    for field in fields:
                        field_key = f"[{field}]"
                        if field_key not in self.all_fields:
                            self.all_fields[field_key] = {
                                "tipo": "texto",
                                "rotulo": field,
                                "obrigatorio": True
                            }
                except Exception as e:
                    print(f"Erro ao extrair campos de {selected_doc}: {e}")
    
    def setup_data_page(self):
        # Limpar widgets anteriores
        for widget in self.data_frame.winfo_children():
            widget.destroy()
        
        # Criar canvas para scrollbar
        canvas = tk.Canvas(self.data_frame)
        scrollbar = ttk.Scrollbar(self.data_frame, orient="vertical", command=canvas.yview)
        scrollable_frame = ttk.Frame(canvas)
        
        scrollable_frame.bind(
            "<Configure>",
            lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
        )
        
        canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
        canvas.configure(yscrollcommand=scrollbar.set)
        
        # Empacotar canvas e scrollbar
        canvas.pack(side="left", fill="both", expand=True, padx=10, pady=10)
        scrollbar.pack(side="right", fill="y")
        
        # Label de instrução
        ttk.Label(scrollable_frame, text="Preencha os campos para os documentos selecionados:", 
                 font=("Arial", 12)).grid(row=0, column=0, columnspan=2, sticky=tk.W, padx=20, pady=(20, 10))
        
        # Adicionar campos agrupados por tipo
        self.field_entries = {}
        row = 1
        
        for field_key, field_info in sorted(self.all_fields.items(), key=lambda x: x[1]['rotulo']):
            label_text = field_info['rotulo']
            if field_info.get('obrigatorio', False):
                label_text += " *"
            
            ttk.Label(scrollable_frame, text=label_text).grid(row=row, column=0, sticky=tk.W, padx=20, pady=5)
            
            if field_info['tipo'] == 'data':
                # Para campos de data, usar um combobox para o dia, mês e ano
                date_frame = ttk.Frame(scrollable_frame)
                
                day_var = tk.StringVar()
                day_combo = ttk.Combobox(date_frame, width=3, textvariable=day_var)
                day_combo['values'] = [str(i).zfill(2) for i in range(1, 32)]
                day_combo.pack(side=tk.LEFT, padx=2)
                
                ttk.Label(date_frame, text="/").pack(side=tk.LEFT)
                
                month_var = tk.StringVar()
                month_combo = ttk.Combobox(date_frame, width=3, textvariable=month_var)
                month_combo['values'] = [str(i).zfill(2) for i in range(1, 13)]
                month_combo.pack(side=tk.LEFT, padx=2)
                
                ttk.Label(date_frame, text="/").pack(side=tk.LEFT)
                
                year_var = tk.StringVar()
                year_combo = ttk.Combobox(date_frame, width=5, textvariable=year_var)
                current_year = datetime.now().year
                year_combo['values'] = [str(i) for i in range(current_year-5, current_year+2)]
                year_combo.pack(side=tk.LEFT, padx=2)
                
                date_frame.grid(row=row, column=1, sticky=tk.W, padx=5, pady=5)
                self.field_entries[field_key] = (day_var, month_var, year_var)
            else:
                # Para campos de texto
                if "Resumo" in field_key or "abstract" in field_key:
                    # Para campos de resumo, usar Text
                    text_widget = tk.Text(scrollable_frame, height=5, width=50)
                    text_widget.grid(row=row, column=1, sticky=tk.W, padx=5, pady=5)
                    self.field_entries[field_key] = text_widget
                else:
                    # Para outros campos de texto
                    text_var = tk.StringVar()
                    ttk.Entry(scrollable_frame, textvariable=text_var, width=50).grid(
                        row=row, column=1, sticky=tk.W, padx=5, pady=5)
                    self.field_entries[field_key] = text_var
            
            row += 1
        
        # Botões para navegar
        btn_frame = ttk.Frame(scrollable_frame)
        btn_frame.grid(row=row, column=0, columnspan=2, pady=20)
        
        ttk.Button(btn_frame, text="Voltar", command=lambda: self.main_notebook.select(0)).pack(side=tk.LEFT, padx=10)
        ttk.Button(btn_frame, text="Avançar para Geração", command=self.proceed_to_generation).pack(side=tk.LEFT, padx=10)
        
        # Botão para carregar dados de JSON
        ttk.Button(btn_frame, text="Carregar Dados de JSON", command=self.load_data_from_json).pack(side=tk.LEFT, padx=10)
        
    def load_data_from_json(self):
        json_file = filedialog.askopenfilename(title="Selecione o arquivo JSON com os dados",
                                               filetypes=[("Arquivos JSON", "*.json")])
        if not json_file:
            return
            
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
                
            # Preencher os campos com os dados do JSON
            for field_key, field_entry in self.field_entries.items():
                if field_key in data:
                    value = data[field_key]
                    
                    # Verificar o tipo de widget/variável
                    if isinstance(field_entry, tuple):  # Data
                        day_var, month_var, year_var = field_entry
                        if isinstance(value, str) and '/' in value:
                            day, month, year = value.split('/')
                            day_var.set(day)
                            month_var.set(month)
                            year_var.set(year)
                    elif isinstance(field_entry, tk.Text):  # Text widget
                        field_entry.delete('1.0', tk.END)
                        field_entry.insert('1.0', value)
                    else:  # StringVar
                        field_entry.set(value)
                        
            messagebox.showinfo("Sucesso", "Dados carregados com sucesso!")
        except Exception as e:
            messagebox.showerror("Erro", f"Erro ao carregar dados: {str(e)}")
    
    def proceed_to_generation(self):
        # Validar campos obrigatórios
        missing_fields = []
        
        for field_key, field_info in self.all_fields.items():
            if field_info.get('obrigatorio', False):
                field_entry = self.field_entries.get(field_key)
                
                if isinstance(field_entry, tuple):  # Data
                    day_var, month_var, year_var = field_entry
                    if not (day_var.get() and month_var.get() and year_var.get()):
                        missing_fields.append(field_info['rotulo'])
                elif isinstance(field_entry, tk.Text):  # Text widget
                    if not field_entry.get('1.0', tk.END).strip():
                        missing_fields.append(field_info['rotulo'])
                elif isinstance(field_entry, tk.StringVar):  # StringVar
                    if not field_entry.get().strip():
                        missing_fields.append(field_info['rotulo'])
        
        if missing_fields:
            messagebox.showwarning("Campos obrigatórios", 
                                  f"Por favor, preencha os seguintes campos obrigatórios:\n- " + 
                                  "\n- ".join(missing_fields))
            return
        
        # Obter valores dos campos
        self.field_values = {}
        
        for field_key, field_entry in self.field_entries.items():
            if isinstance(field_entry, tuple):  # Data
                day_var, month_var, year_var = field_entry
                date_value = f"{day_var.get()}/{month_var.get()}/{year_var.get()}"
                self.field_values[field_key] = date_value
            elif isinstance(field_entry, tk.Text):  # Text widget
                text_value = field_entry.get('1.0', tk.END).strip()
                self.field_values[field_key] = text_value
            else:  # StringVar
                self.field_values[field_key] = field_entry.get()
        
        # Configurar página de geração
        self.setup_gen_page()
        
        # Mudar para a próxima aba
        self.main_notebook.select(2)
    
    def setup_gen_page(self):
        # Limpar widgets anteriores
        for widget in self.gen_frame.winfo_children():
            widget.destroy()
        
        # Label de instrução
        ttk.Label(self.gen_frame, text="Revise os dados antes de gerar os documentos:", 
                 font=("Arial", 12)).pack(anchor=tk.W, padx=20, pady=(20, 10))
        
        # Frame para mostrar resumo dos dados
        summary_frame = ttk.Frame(self.gen_frame)
        summary_frame.pack(fill=tk.BOTH, expand=True, padx=20, pady=10)
        
        # Criar canvas com scrollbar
        canvas = tk.Canvas(summary_frame)
        scrollbar = ttk.Scrollbar(summary_frame, orient="vertical", command=canvas.yview)
        scrollable_frame = ttk.Frame(canvas)
        
        scrollable_frame.bind(
            "<Configure>",
            lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
        )
        
        canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
        canvas.configure(yscrollcommand=scrollbar.set)
        
        # Empacotar canvas e scrollbar
        canvas.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")
        
        # Mostrar os documentos selecionados
        ttk.Label(scrollable_frame, text="Documentos a serem gerados:", 
                 font=("Arial", 10, "bold")).grid(row=0, column=0, columnspan=2, sticky=tk.W, pady=(0, 5))
        
        for i, doc in enumerate(self.selected_docs):
            ttk.Label(scrollable_frame, text=f"• {doc}").grid(row=i+1, column=0, columnspan=2, sticky=tk.W)
        
        # Mostrar os principais campos preenchidos
        row = len(self.selected_docs) + 2
        ttk.Label(scrollable_frame, text="Dados principais:", 
                 font=("Arial", 10, "bold")).grid(row=row, column=0, columnspan=2, sticky=tk.W, pady=(10, 5))
        row += 1
        
        # Mostrar apenas alguns campos chave para não sobrecarregar a tela
        key_fields = ["[nome do aluno]", "[título]", "[data defesa]", "[nome orientador]"]
        for field in key_fields:
            if field in self.field_values:
                label_text = next((info['rotulo'] for key, info in self.all_fields.items() if key == field), field)
                ttk.Label(scrollable_frame, text=f"{label_text}:").grid(row=row, column=0, sticky=tk.W, padx=(10, 5))
                ttk.Label(scrollable_frame, text=self.field_values[field]).grid(row=row, column=1, sticky=tk.W)
                row += 1
        
        # Frame para botões
        btn_frame = ttk.Frame(self.gen_frame)
        btn_frame.pack(pady=20)
        
        ttk.Button(btn_frame, text="Voltar para Edição", 
                  command=lambda: self.main_notebook.select(1)).pack(side=tk.LEFT, padx=10)
        
        ttk.Button(btn_frame, text="Salvar Dados como JSON", 
                  command=self.save_data_to_json).pack(side=tk.LEFT, padx=10)
        
        ttk.Button(btn_frame, text="Gerar Documentos", 
                  command=self.generate_documents).pack(side=tk.LEFT, padx=10)
    
    def save_data_to_json(self):
        file_path = filedialog.asksaveasfilename(defaultextension=".json", 
                                              filetypes=[("Arquivos JSON", "*.json")],
                                              title="Salvar dados como")
        if not file_path:
            return
            
        try:
            with open(file_path, 'w', encoding='utf-8') as f:
                json.dump(self.field_values, f, ensure_ascii=False, indent=4)
            messagebox.showinfo("Sucesso", "Dados salvos com sucesso!")
        except Exception as e:
            messagebox.showerror("Erro", f"Erro ao salvar dados: {str(e)}")
    
    def generate_documents(self):
        # Pedir diretório para salvar os documentos gerados
        output_dir = filedialog.askdirectory(title="Selecione o diretório para salvar os documentos gerados")
        if not output_dir:
            return
        
        try:
            # Processar cada documento selecionado
            for doc_name in self.selected_docs:
                doc_path = os.path.join(self.docx_dir, doc_name)
                
                # Carregar o documento
                doc = Document(doc_path)
                
                # Substituir os campos nos parágrafos
                for para in doc.paragraphs:
                    for field_key, field_value in self.field_values.items():
                        if field_key in para.text:
                            para.text = para.text.replace(field_key, field_value)
                
                # Substituir os campos nas tabelas
                for table in doc.tables:
                    for row in table.rows:
                        for cell in row.cells:
                            for paragraph in cell.paragraphs:
                                for field_key, field_value in self.field_values.items():
                                    if field_key in paragraph.text:
                                        paragraph.text = paragraph.text.replace(field_key, field_value)
                
                # Definir nome do arquivo de saída
                base_name = os.path.splitext(doc_name)[0]
                nome_aluno = self.field_values.get("[nome do aluno]", "").replace(" ", "_")
                if not nome_aluno:
                    nome_aluno = datetime.now().strftime("%Y%m%d_%H%M%S")
                
                output_filename = f"{base_name}_{nome_aluno}.docx"
                output_path = os.path.join(output_dir, output_filename)
                
                # Salvar o documento
                doc.save(output_path)
            
            messagebox.showinfo("Sucesso", 
                             f"Todos os {len(self.selected_docs)} documentos foram gerados com sucesso!")
        except Exception as e:
            messagebox.showerror("Erro", f"Erro ao gerar documentos: {str(e)}")

if __name__ == "__main__":
    root = tk.Tk()
    app = DocumentFillerApp(root)
    root.mainloop()